# web scraping RIP.ie with beautifulsoup

In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datefinder

def get_dn_page(n):
    """Get death notice text from page matching the id number"""
    
    url = 'https://rip.ie/showdn.php?dn=%s' %n

    user_agent = 'Chrome/107.0.5304.110'
    headers = {'User-Agent': user_agent}
    req = requests.get(url,headers=headers)   
    soup = BeautifulSoup(req.content, 'html.parser')
    if soup.title == None:
        title = ''
    else:        
        title  = soup.title.text.strip()
    name=''    
    for s in ['Death Notice of ','The death has occurred of ']:
        if title.startswith(s):
            name = title.split(s)[1]   
    elem = soup.find_all("div", id="dn_photo_and_text")
    
    if len(elem) == 0:
        return name, '', '', '', ''
    rows = elem[0].find_all('p')
    if len(rows) == 0:
        rows = elem[0].find_all('td')
    text = ';'.join([r.text.strip() for r in rows]).replace('\n','')
    #address
    addrelem = soup.find("span", class_='small_addr') 
    if addrelem != None:
        address = addrelem.text.strip()
    else:
        address = ''
    #county  
    ctyelem = soup.find("li", class_='fd_county') 
    if ctyelem != None:
        county = ctyelem.text.strip()
    else:
        county = ''
    #date
    dateelem = soup.find("div", class_='ddeath')
    if dateelem == None:
        dateelem = soup.find("div", class_='dpubl')
    s = dateelem.text.strip()
    try:
        date = list(datefinder.find_dates(s))[0]
    except:
        date = ''
    print (n, date, name, address, county)
    return name, date, county, address, text

In [2]:
def get_dn_page_new(n):
    """Get death notice text from page matching the id number"""
    
    url = 'https://rip.ie/showdn.php?dn=%s' %n
    print (url)
    user_agent = 'Chrome/107.0.5304.110'
    headers = {'User-Agent': user_agent}
    req = requests.get(url,headers=headers)   
    soup = BeautifulSoup(req.content, 'html.parser')
   
    elem = soup.select('[class*=DeathNotice_person-name]')
    print (elem)
    if len(elem) == 0:
        return '', '', '', '', ''
    else:
        name = elem[0].get_text(strip=True)
    
    elem = soup.select('[class*=DeathNotice_dates-death-date]')    
    if len(elem) == 0:
        print (n, name, 'no date')
        return '', '', '', '', ''
    else:
        date = elem[0].get_text(strip=True)    
    
    date = list(datefinder.find_dates(date))[0]

    address = soup.select('[class*=DeathNotice_tags-item]')[0].get_text(strip=True)
    #print (address) 
    county = address.split()[-1]

    desc = soup.select('[class*=DeathNotice_description]')
    text = []
    for elem in desc:
        text.append(elem.get_text(strip=True))
    text = ' '.join(text)
    #print (text)
   
    print (n, date, name, address, county)
    return name, date, county, address, text


In [3]:
get_dn_page_new(540710)

https://rip.ie/showdn.php?dn=540710
[]


('', '', '', '', '')

In [2]:
df = pd.read_pickle('rip_dn_scrape.pkl')
len(df)

539492

In [3]:
df[-3:]

,id,name,date,address,notice,county
539489,540708,Pat Lynch,2023-07-02 00:00:00,"Drumgore, Loughduff, Cavan","Lynch Pat, Drumgore, Loughduff, Co. Cavan, Sun...",Cavan
539490,540709,Josephine BAILEY(née Monaghan),2023-07-04 00:00:00,"Clonfanlough, Offaly",Josephine Bailey (née Monaghan)​​​​Clonfanloug...,Offaly
539491,540710,Cheri O'Mathuna,2023-07-02 00:00:00,"Millfarm, Dunboyne, Meath","Cheri O’Mathuna, 64, formerly of Dunboyne, Co....",Meath


## iterate over a range of ids and get info

In [ ]:
#read current table in so we skip those already done
df = pd.read_pickle('rip_dn_scrape.pkl')
#df = pd.read_parquet('rip_dn_scrape.parquet')
print (len(df))
ids = list(df.id)

results={}
for n in range(539000,541000):
    if n in ids:
        continue
    name,date,cty,addr,txt = get_dn_page_new(n)   
    if name == '':
        continue
    results[n] = [name,date,cty,addr,txt]
    time.sleep(0.05)

In [166]:
res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice']).reset_index()
res = res.rename(columns={'index':'id'})

In [167]:
res

,id,name,date,county,address,notice
0,540101,Desmond Morrissey,2023-06-27,Dublin,"Dublin 12, Dublin","Morrissey Desmond (Dessie) (27th June 2023), l..."
1,540102,Catherine (Kala) Monks,2023-06-27,Dublin,"Beaumont, Dublin / Inchicore, Dublin",Monks Catherine (Kala) (Beaumont and formerly ...
2,540103,Raymond Crowley,2023-06-27,Cork,"Carrigtwohill, Cork","Crowley, Carrigtwohill and Ard Dara, Cope Foun..."
3,540104,Eamonn McMAHON,2023-06-27,Dublin,"Palmerstown, Dublin","McMAHON, Eamonn, 27th June 2023 (late of Kenn..."
4,540106,Bernard Lynch,2023-06-28,Galway,"Hopsons Lane & Jubilee Street, Ballinasloe, Ga...",Bernard passed peacefully in the wonderful car...
...,...,...,...,...,...,...
583,540706,Brendan Mc Hugh,2023-07-04,Cavan,"Clontarf, Dublin / Swanlinbar, Cavan",Predeceased by his parents Patrick and Mary El...
584,540707,Marie (Gertrude) Coleman(née Burns),2023-07-03,Galway,"Laurencetown, Ballinasloe, Galway",The death has occurred of Marie (Gertrude) Col...
585,540708,Pat Lynch,2023-07-02,Cavan,"Drumgore, Loughduff, Cavan","Lynch Pat, Drumgore, Loughduff, Co. Cavan, Sun..."
586,540709,Josephine BAILEY(née Monaghan),2023-07-04,Offaly,"Clonfanlough, Offaly",Josephine Bailey (née Monaghan)​​​​Clonfanloug...


In [168]:
new = pd.concat([df,res]).reset_index(drop=True)
new=new[~new.id.duplicated(keep='first')]
#new=new.replace('',None).dropna(axis=0,subset=['date'])
#new['date'] = pd.to_datetime(new['date']).apply(lambda x: x.strftime('%d/%m/%Y'))
print (len(df),len(res),len(new))
#new.to_parquet('rip_dn_scrape.parquet')
new.to_pickle('rip_dn_scrape.pkl')

538904 588 539492


## clean data

In [169]:
x=new
print (len(x))
x=x.replace('',None).dropna(axis=0,subset=['date'])
x['date'] = pd.to_datetime(x['date'],format='mixed')#.apply(lambda x: x.strftime('%d/%m/%Y'))
x=x.drop_duplicates(['name','notice'])
x=x.drop_duplicates(['name','address'])
x=x.drop_duplicates(['name','date','county'])
x = x[~x.address.isnull()]
nc = ['Fermanagh','Armagh','Tyrone','Down','Antrim','Derry']
x = x[~x.county.isin(nc)]
x = x[~x.address.str.contains('|'.join(nc))]
x=x.sort_values('id')
print (len(x))
#x.to_pickle('rip_dn_scrape_processed.pkl')
x.to_parquet('rip_dn_scrape_processed.parquet')

539492
502295


In [25]:
#read pasted data
f = open('rip_pasted.txt')
new = []
i=0
id=1000
lines = list(f.readlines())
for l in lines:    
    print (l)
    

Adams

, 

Anthony (Anto)

Ballinteer

Dublin

03/07/23

BARRY

, 

TED

Carrigaline

Cork

03/07/23

Beirne (O'Beirne)

, 

Maura

Coolock

Dublin

03/07/23

Beirne (O'Beirne)

, 

Maura

Stranorlar

Donegal

03/07/23

Boland

(née Williams), 

Lorraine

Limerick City

Limerick

03/07/23

Boyle

, 

Deboragh

Cavan Town

Cavan

03/07/23

Boyle

(née Kelly), 

Madge

Glenties

Donegal

03/07/23

Bradshaw

, 

Henry (Harry)

Navan Road

Dublin

03/07/23

Brady

, 

Enda

Kells

Meath

03/07/23

Brady

, 

Enda

Virginia

Cavan

03/07/23

Brandley

, 

Eugene (Jimmy)

Williamstown

Galway

03/07/23

Brandley

, 

Eugene (Jimmy)

Kinlough

Leitrim

03/07/23

Brickley

, 

Jim

Clontarf

Dublin

03/07/23

Browne

(née Mooney), 

Alice

Tallaght

Dublin

03/07/23

Browne

(née Mooney), 

Alice

Drimnagh

Dublin

03/07/23

Buckley

, 

Johnny

Pallasgreen

Limerick

03/07/23

Burke

(née Gough), 

Doris

Corrandulla

Galway

03/07/23

Burke

(née Gough), 

Doris

Inchicore

Dublin

03/07/23
